<a href="https://colab.research.google.com/github/afujii/NLTK/blob/master/RDFLib%E7%99%BE%E4%BA%BA%E4%B8%80%E9%A6%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RDFLibと百人一種
はじめに、ドライブをマウントして、CSVファイルを読み込む。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/MyDrive/Colab Notebooks/hyakuninUTF8.txt',encoding='UTF8') as fp:
  utas = fp.readlines()

ライブラリを用意しRDFLib のグラフを作成する。

In [5]:
!pip install rdflib

     |████████████████████████████████| 407 kB 5.1 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 


In [6]:
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import FOAF

g = Graph()

ライブラリを用意し、グラフにデータを読み込む。　<br>
今回の例で使用した　s, p, o の関係は以下。　<br>
|yomiteID -FOAF.name-> Literal() <br>
 yomiteID -utaid-> utaID <br>
 utaID -utaliteral-> Literal() <br>
 utaID -kami-> kamiID <br>
 utaID -simo-> simoID <br>
 kamiID -kanji-> Literal <br>
 simoID -kanji-> Literal <br>
 kamiID -kana-> Literal <br>
simoID -kana-> Litral <br>

In [7]:
head  = utas.pop(0)# utas は、No.1 からの全データ（list）になる。

yomite = [ uta.split(',')[1] for uta in utas ] #読み手
kamikanji = [ uta.split(',')[2] for uta in utas ] #上の句（漢字）
simokanji = [ uta.split(',')[3] for uta in utas ] #下の句（漢字）
kamikana = [ uta.split(',')[4] for uta in utas ] #上の句（かな）
simokana = [ uta.split(',')[5] for uta in utas ] #下の句（かな）

############################################
ns = Namespace("http://hyakunin.com#")
yomiteIDs = [URIRef( "ID"+str(i).zfill(3))  for i in range(1,101)]
utaIDs = [ URIRef("utaID"+str(i).zfill(3))  for i in range(1,101)]
kamiIDs = [ URIRef("kamiID"+str(i).zfill(3))  for i in range(1,101)]
simoIDs = [ URIRef("simoID"+str(i).zfill(3))  for i in range(1,101)]

for i in range(0,100):
  g.add((yomiteIDs[i],FOAF.name,Literal(yomite[i])))
  g.add((yomiteIDs[i],ns['utaid'],utaIDs[i]))
  #####################
  g.add((utaIDs[i],ns['utaliteral'],Literal(kamikanji[i]+simokanji[i])))
  g.add((utaIDs[i],ns['kami'],kamiIDs[i]))
  g.add((utaIDs[i],ns['simo'],simoIDs[i]))
  #####################
  g.add((kamiIDs[i],ns['kanji'],Literal(kamikanji[i])))
  g.add((kamiIDs[i],ns['kana'],Literal(kamikana[i])))
  g.add((simoIDs[i],ns['kanji'],Literal(simokanji[i])))
  g.add((simoIDs[i],ns['kana'],Literal(simokana[i])))



In [8]:
######### 以上で　g の準備ができた
# 視覚化 百人一首全体では、大きいので「天智天皇」のみ

newg = Graph()
for s, p, o in g.triples((URIRef("ID001"), None, None)):
  newg.add((s,p,o))
for s, p, o in g.triples((URIRef("utaID001"),None,None)):
  newg.add((s,p,o))
############## 共通項の utaID001 は、自動的に共有される

In [ ]:
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

G = rdflib_to_networkx_multidigraph(newg)# 部分グラフ newg 

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw(G, with_labels=True)
plt.show()


In [ ]:
############ 別の視覚化ライブラリを試す
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()
    display(Image(png))

visualize(newg)

## SPARQLクエリの適用
作成したグラフに対して、クエリ言語を利用して検索する例を示していく。

In [12]:
query ="""
SELECT DISTINCT ?name  ?uta
WHERE {
    ?yid foaf:name ?name .
    ?yid ns:utaid ?uid .
    ?uid ns:utaliteral  ?uta .
}
"""
qres = g.query(query,initNs={ "ns":ns , "foaf":FOAF})
for row in qres:
  print(f"{ row.name}:{row.uta}")

天智天皇:秋の田のかりほの庵の苫をあらみわが衣手は露にぬれつつ
持統天皇:春過ぎて夏来にけらし白妙の衣干すてふ天の香具山
柿本人麻呂:あしびきの山鳥の尾のしだり尾のながながし夜をひとりかも寝む
山辺赤人:田子の浦にうち出でてみれば白妙の富士の高嶺に雪は降りつつ
猿丸大夫:奥山に紅葉踏み分け鳴く鹿の声聞く時ぞ秋は悲しき
中納言家持:かささぎの渡せる橋に置く霜の白きを見れば夜ぞ更けにける
安倍仲麻呂:天の原ふりさけみれば春日なる三笠の山に出でし月かも
喜撰法師:わが庵は都の辰巳しかぞ住む世をうぢ山と人はいふなり
小野小町:花の色は移りにけりないたづらにわが身世にふるながめせしまに
蝉丸:これやこの行くも帰るも別れては知るも知らぬも逢坂の関
参議篁:わたの原八十島かけて漕ぎ出でぬと人にはつげよ海人の釣船
僧正遍昭:天つ風雲の通ひ路吹きとぢよ乙女の姿しばしとどめむ
陽成院:筑波嶺の峰より落つるみなの川恋ぞつもりて淵となりぬる
河原左大臣:陸奥のしのぶもぢずりたれゆえに乱れそめにしわれならなくに
光孝天皇:君がため春の野に出でて若菜摘むわが衣手に雪は降りつつ
中納言行平:立ち別れいなばの山の峰に生ふるまつとし聞かば今帰り来む
在原業平朝臣:ちはやぶる神代も聞かず竜田川から紅に水くくるとは
藤原敏行朝臣:住の江の岸に寄る波よるさへや夢の通ひ路人目よくらむ
伊勢:難波潟短き葦のふしの間も逢はでこの世を過ぐしてよとや
元良親王:わびぬれば今はた同じ難波なる身をつくしても逢はむとぞ思ふ
素性法師:今来むといひしばかりに長月の有明の月を待ち出でつるかな
文屋康秀:吹くからに秋の草木のしをるればむべ山風を嵐といふらむ
大江千里:月見れば千々にものこそ悲しけれわが身ひとつの秋にはあらねど
菅家:このたびは幣もとりあへず手向山紅葉の錦神のまにまに
三条右大臣:名にし負はば逢坂山のさねかずら人に知られでくるよしもがな
貞信公:小倉山峰のもみぢ葉心あらば今ひとたびのみゆき待たなむ
中納言兼輔:みかの原わきて流るる泉川いつ見きとてか恋しかるらむ
源宗于朝臣:山里は冬ぞ寂しさまさりける人目も草もかれぬと思へば
凡河内躬恒:心あてに折らばや折らむ初霜の置きまどはせる白菊の花
壬生忠岑:有明のつれなく見えし別れより暁ばかり憂きものはなし
坂上是則:朝ぼらけ有明の月と見るまでに吉野の里に降れる白雪
春道列